# Sample 10 Random Tamil Units of Analysis for Human Evaluation

This notebook randomly selects 10 units (verses/couplets) from Tamil texts and displays their top 3 moral foundations based on embedding-based scoring.

## Texts Included:
- Thirukkural
- Aathichoodi
- Moothurai
- Nalvazhi
- Ulakaneethi
- Konraiventhan
- Vivekacinthamani

In [1]:
import pandas as pd
import numpy as np
import random
import glob
import os
from IPython.display import display, Markdown

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

## Step 1: Load All Tamil Texts and Moral Scores

In [2]:
# Get all CSV files from tamil-csv folder
csv_files = glob.glob('tamil-csv/*.csv')
csv_files.sort()

print("Found CSV files:")
for f in csv_files:
    print(f"  - {os.path.basename(f)}")

# Dictionary to store all data
texts_data = {}
embedding_data = {}

# Load each text
for csv_file in csv_files:
    basename = os.path.basename(csv_file).replace('_cleaned.csv', '')
    
    # Load original text
    df = pd.read_csv(csv_file)
    texts_data[basename] = df
    
    # Load corresponding embedding results
    embedding_file = f'tamil-step3-results/{basename}_cleaned_moral_scores.csv'
    if os.path.exists(embedding_file):
        embedding_df = pd.read_csv(embedding_file)
        embedding_data[basename] = embedding_df
        print(f"✓ {basename}: {len(df)} texts loaded with embedding results")
    else:
        print(f"⚠ {basename}: {len(df)} texts loaded (NO embedding results found)")

print(f"\nTotal texts loaded: {len(texts_data)}")
print(f"Total with embedding results: {len(embedding_data)}")

Found CSV files:
  - aathichoodi_cleaned.csv
  - konraiventhan_cleaned.csv
  - moothurai_cleaned.csv
  - nalvazhi_cleaned.csv
  - thirukkural_cleaned.csv
  - ulakaneethi_cleaned.csv
  - vivekacinthamani_cleaned.csv
✓ aathichoodi: 111 texts loaded with embedding results
✓ konraiventhan: 93 texts loaded with embedding results
✓ moothurai: 30 texts loaded with embedding results
✓ nalvazhi: 40 texts loaded with embedding results
✓ thirukkural: 1334 texts loaded with embedding results
✓ ulakaneethi: 13 texts loaded with embedding results
✓ vivekacinthamani: 135 texts loaded with embedding results

Total texts loaded: 7
Total with embedding results: 7


## Step 2: Collect All Units of Analysis

In [3]:
# Collect all units from all texts
all_units = []

for text_name in texts_data.keys():
    if text_name not in embedding_data:
        continue
    
    texts_df = texts_data[text_name]
    emb_df = embedding_data[text_name]
    
    # Determine which column contains the text
    text_column = None
    for col in ['couplet', 'text', 'verse', 'saying', 'line']:
        if col in texts_df.columns:
            text_column = col
            break
    
    if text_column is None:
        # Use the last column as text
        text_column = texts_df.columns[-1]
    
    # Get ID column
    id_column = None
    for id_col in ['kural_number', 'line_number', 'verse_number', 'number', 'id']:
        if id_col in texts_df.columns:
            id_column = id_col
            break
    
    # Collect units from this text
    for idx in range(len(texts_df)):
        unit_id = texts_df.iloc[idx][id_column] if id_column else idx + 1
        original_text = texts_df.iloc[idx][text_column]
        
        # Get embedding scores
        emb_row = emb_df.iloc[idx]
        moral_cols = [col for col in emb_df.columns if '.' in col and col not in ['dominant_moral', 'dominant_score']]
        emb_scores = {col: emb_row[col] for col in moral_cols}
        emb_top3 = sorted(emb_scores.items(), key=lambda x: x[1], reverse=True)[:3]
        
        all_units.append({
            'text_name': text_name,
            'unit_id': unit_id,
            'text': original_text,
            'top3_morals': emb_top3
        })

print(f"Total units collected: {len(all_units)}")

Total units collected: 1756


## Step 3: Randomly Select 10 Units

In [4]:
# Randomly select 10 samples (or all if less than 10)
num_samples = min(10, len(all_units))
selected_samples = random.sample(all_units, num_samples)

print("="*80)
print(f"10 RANDOMLY SELECTED TAMIL UNITS OF ANALYSIS")
print("="*80)
print(f"\nTotal units available: {len(all_units)}")
print(f"Displaying {num_samples} random samples\n")

10 RANDOMLY SELECTED TAMIL UNITS OF ANALYSIS

Total units available: 1756
Displaying 10 random samples



## Step 4: Display Results in Table Format

In [5]:
# Create markdown table
print("\n## Random Tamil Units with Top 3 Moral Foundations\n")
print("| Unit ID | Text (Tamil) | Top 3 Moral Foundations |")
print("|---------|--------------| ------------------------|")

for sample in selected_samples:
    # Unit ID
    unit_id = f"{sample['text_name']}-{sample['unit_id']}"
    
    # Text (full text, not truncated)
    text_display = sample['text'].replace('\n', ' ').replace('|', '\\|')
    
    # Top 3 morals - formatted as numbered list
    morals_str = "<br>".join([f"{i}. {moral}: {score:.3f}" 
                              for i, (moral, score) in enumerate(sample['top3_morals'], 1)])
    
    # Print row
    print(f"| {unit_id} | {text_display} | {morals_str} |")

print("\n" + "="*80)
print("END OF SAMPLE ANALYSIS")
print("="*80)


## Random Tamil Units with Top 3 Moral Foundations

| Unit ID | Text (Tamil) | Top 3 Moral Foundations |
|---------|--------------| ------------------------|
| thirukkural-1036 | தொழுதுண்டு பின்செல் பவர். பலகுடை நீழலும் தங்குடைக்கீழ்க் காண்பர் | 1. authority.virtue: 0.503<br>2. sanctity.virtue: 0.491<br>3. sanctity.vice: 0.484 |
| moothurai-25 | நஞ்சுடைமை தானறிந்து நாகம் கரந்துறையும் அஞ்சாப் புறங்கிடக்கும் நீர்ப்பாம்பு - நெஞ்சில் கரவுடையார் தம்மைக் கரப்பர் கரவார் கரவிலா நெஞ்சத் தவர். | 1. sanctity.vice: 0.624<br>2. sanctity.virtue: 0.543<br>3. authority.virtue: 0.514 |
| aathichoodi-52 | செய்வன திருந்தச் செய். | 1. sanctity.virtue: 0.526<br>2. care.virtue: 0.516<br>3. authority.virtue: 0.503 |
| thirukkural-1245 | நினைத்தொன்று சொல்லாயோ நெஞ்சே எனைத்தொன்றும் எவ்வநோய் தீர்க்கும் மருந்து. | 1. sanctity.virtue: 0.515<br>2. care.virtue: 0.511<br>3. sanctity.vice: 0.469 |
| thirukkural-290 | மற்றைய தேற்றா தவர். கள்வார்க்குத் தள்ளும் உயிர்நிலை கள்வார்க்குத் | 1. fairness.vice: 0.669<br>2. loy

## Step 5: Display as Detailed List (Easier to Read)

In [6]:
print("\n" + "="*80)
print("DETAILED VIEW: 10 RANDOM TAMIL UNITS")
print("="*80)

for i, sample in enumerate(selected_samples, 1):
    print(f"\n{'='*80}")
    print(f"SAMPLE {i}/{num_samples}")
    print(f"{'='*80}")
    print(f"Text: {sample['text_name'].replace('_', ' ').title()}")
    print(f"Unit ID: {sample['unit_id']}")
    print(f"\nOriginal Text (Tamil):")
    print(f"  {sample['text']}")
    print(f"\nTop 3 Moral Foundations:")
    for rank, (moral, score) in enumerate(sample['top3_morals'], 1):
        # Split moral into foundation and virtue/vice
        if '.' in moral:
            foundation, aspect = moral.split('.')
            print(f"  {rank}. {foundation.capitalize()}.{aspect.capitalize():<10s} : {score:.4f}")
        else:
            print(f"  {rank}. {moral:<20s} : {score:.4f}")

print(f"\n{'='*80}")
print("END OF DETAILED ANALYSIS")
print(f"{'='*80}")


DETAILED VIEW: 10 RANDOM TAMIL UNITS

SAMPLE 1/10
Text: Thirukkural
Unit ID: 1036

Original Text (Tamil):
  தொழுதுண்டு பின்செல் பவர். பலகுடை நீழலும் தங்குடைக்கீழ்க் காண்பர்

Top 3 Moral Foundations:
  1. Authority.Virtue     : 0.5032
  2. Sanctity.Virtue     : 0.4906
  3. Sanctity.Vice       : 0.4839

SAMPLE 2/10
Text: Moothurai
Unit ID: 25

Original Text (Tamil):
  நஞ்சுடைமை தானறிந்து நாகம் கரந்துறையும் அஞ்சாப் புறங்கிடக்கும் நீர்ப்பாம்பு - நெஞ்சில் கரவுடையார் தம்மைக் கரப்பர் கரவார் கரவிலா நெஞ்சத் தவர்.

Top 3 Moral Foundations:
  1. Sanctity.Vice       : 0.6241
  2. Sanctity.Virtue     : 0.5432
  3. Authority.Virtue     : 0.5142

SAMPLE 3/10
Text: Aathichoodi
Unit ID: 52

Original Text (Tamil):
  செய்வன திருந்தச் செய்.

Top 3 Moral Foundations:
  1. Sanctity.Virtue     : 0.5260
  2. Care.Virtue     : 0.5157
  3. Authority.Virtue     : 0.5026

SAMPLE 4/10
Text: Thirukkural
Unit ID: 1245

Original Text (Tamil):
  நினைத்தொன்று சொல்லாயோ நெஞ்சே எனைத்தொன்றும் எவ்வநோய் தீர்க்கும் மருந்து.


## Step 6: Save to CSV

In [7]:
# Create table data for CSV export
table_data = []
for sample in selected_samples:
    table_data.append({
        'Text_Name': sample['text_name'].replace('_', ' ').title(),
        'Unit_ID': sample['unit_id'],
        'Tamil_Text': sample['text'],
        '1st_Moral': sample['top3_morals'][0][0],
        '1st_Score': f"{sample['top3_morals'][0][1]:.4f}",
        '2nd_Moral': sample['top3_morals'][1][0] if len(sample['top3_morals']) > 1 else '',
        '2nd_Score': f"{sample['top3_morals'][1][1]:.4f}" if len(sample['top3_morals']) > 1 else '',
        '3rd_Moral': sample['top3_morals'][2][0] if len(sample['top3_morals']) > 2 else '',
        '3rd_Score': f"{sample['top3_morals'][2][1]:.4f}" if len(sample['top3_morals']) > 2 else ''
    })

samples_df = pd.DataFrame(table_data)
samples_df.to_csv('10_random_tamil_units.csv', index=False, encoding='utf-8')
print("✓ Saved to: 10_random_tamil_units.csv")

# Display the dataframe
print("\nPreview of saved data:")
print(samples_df.to_string(index=False))

✓ Saved to: 10_random_tamil_units.csv

Preview of saved data:
  Text_Name  Unit_ID                                                                                                                                   Tamil_Text        1st_Moral 1st_Score       2nd_Moral 2nd_Score        3rd_Moral 3rd_Score
Thirukkural     1036                                                                             தொழுதுண்டு பின்செல் பவர். பலகுடை நீழலும் தங்குடைக்கீழ்க் காண்பர் authority.virtue    0.5032 sanctity.virtue    0.4906    sanctity.vice    0.4839
  Moothurai       25 நஞ்சுடைமை தானறிந்து நாகம் கரந்துறையும் அஞ்சாப் புறங்கிடக்கும் நீர்ப்பாம்பு - நெஞ்சில் கரவுடையார் தம்மைக் கரப்பர் கரவார் கரவிலா நெஞ்சத் தவர்.    sanctity.vice    0.6241 sanctity.virtue    0.5432 authority.virtue    0.5142
Aathichoodi       52                                                                                                                       செய்வன திருந்தச் செய்.  sanctity.virtue    0.5260     care.virtue    0.51